In [1]:
dataset1 = []

class DataPoint:
  def __init__(self, status, vars):
    self.isTrue = status
    self.title = vars[0]
    self.text = vars[1]
    self.subject = vars[2]
    self.data = vars[3]

  def print(self):
    print(self.isTrue + " - " + self.title + " - " + self.text)
    



In [10]:
import pandas as pd

dfFalse = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = pd.DataFrame()
ds1["text"] = dfFalse["text"]
ds1.insert(0, 'label', 'fake')
dfTrue = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = pd.DataFrame()
ds2["text"] = dfTrue["text"]
ds2.insert(0, 'label', 'true')
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(dataset1)


      label                                               text
0      fake  Donald Trump just couldn t wish all Americans ...
1      fake  House Intelligence Committee Chairman Devin Nu...
2      fake  On Friday, it was revealed that former Milwauk...
3      fake  On Christmas day, Donald Trump announced that ...
4      fake  Pope Francis used his annual Christmas Day mes...
5      fake  The number of cases of cops brutalizing and ki...
6      fake  Donald Trump spent a good portion of his day a...
7      fake  In the wake of yet another court decision that...
8      fake  Many people have raised the alarm regarding th...
9      fake  Just when you might have thought we d get a br...
10     fake  A centerpiece of Donald Trump s campaign, and ...
11     fake  Republicans are working overtime trying to sel...
12     fake  Republicans have had seven years to come up wi...
13     fake  The media has been talking all day about Trump...
14     fake  Abigail Disney is an heiress with brass ov

In [3]:
dfTrue2 = pd.read_csv (r'Databases/Data1/True.csv')
ds22 = pd.DataFrame()
ds22["text"] = dfTrue2["text"]
ds22.insert(0, 'label', 'true')
ds22['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [4]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_lg
!{sys.executable} -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
LGVars = pd.DataFrame()
import numpy as np
import spacy as sp
import pandas as pd

# Load English tokenizer, tagger, parser and NER
nlp = sp.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []
for t in doc:
    row = [t.text, t.lemma_, t.pos_, sp.explain(t.pos_), t.is_stop]
    rows.append(row)
df = pd.DataFrame(rows, columns=cols)
print(df)

         text      lemma    POS                    explain  stopword
0        When       when  SCONJ  subordinating conjunction      True
1   Sebastian  Sebastian  PROPN                proper noun     False
2       Thrun      Thrun  PROPN                proper noun     False
3     started      start   VERB                       verb     False
4     working       work   VERB                       verb     False
5          on         on    ADP                 adposition      True
6        self       self   NOUN                       noun     False
7           -          -  PUNCT                punctuation     False
8     driving      drive   VERB                       verb     False
9        cars        car   NOUN                       noun     False
10         at         at    ADP                 adposition      True
11     Google     Google  PROPN                proper noun     False
12         in         in    ADP                 adposition      True
13       2007       2007    NUM   

In [6]:
import sys
!{sys.executable} -m pip install xgboost

In [12]:
import pandas as pd

dfFalse = pd.read_csv (r'Databases/Data1/Fake.csv')
ds1 = pd.DataFrame()
ds1["text"] = dfFalse["text"]
ds1.insert(0, 'label', 'fake')
dfTrue = pd.read_csv (r'Databases/Data1/True.csv')
ds2 = pd.DataFrame()
ds2["text"] = dfTrue["text"]
ds2.insert(0, 'label', 'true')
dataset1 = pd.concat([ds1, ds2])
dataset1.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# Import libraries
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

print(dataset1.head(5))

nltk.download('stopwords')

def text_preprocess(message):
    # Remove punctuations
    nopunc = [char for char in message if char not in string.punctuation]

    # Join the characters again
    nopunc = "".join(nopunc)
    nopunc = nopunc.lower()

    # Remove any stopwords and characters that arent letters
    nostop = [
        word
        for word in nopunc.split()
        if word.lower() not in stopwords.words("english") and word.isalpha()
    ]
    return nostop


false_text = dataset1[dataset1["label"] == "fake"]["text"]
true_text = dataset1[dataset1["label"] == "true"]["text"]
print(f"Number of fake messages: {len(false_text)}")
print(f"Number of true messages: {len(true_text)}")



  label                                               text
0  fake  Donald Trump just couldn t wish all Americans ...
1  fake  House Intelligence Committee Chairman Devin Nu...
2  fake  On Friday, it was revealed that former Milwauk...
3  fake  On Christmas day, Donald Trump announced that ...
4  fake  Pope Francis used his annual Christmas Day mes...
Number of fake messages: 23452
Number of true messages: 21417


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jjfri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
####################################################################### TAKES FOREVER TO PROSESS
from sklearn.utils import shuffle
data_shuffled = dataset1.sample(frac=1).reset_index(drop=True)
print(data_shuffled.head(10))
#data_shuffled = dataset1.sample(len(dataset1))

# 0.0878 seconds per row // 89.922 min for entire dataset

# Remove punctuations/stopwords from all messages
data_shuffled["text"] = data_shuffled["text"].apply(text_preprocess)
data_shuffled.head(20)



  label                                               text
0  true  BEIJING (Reuters) - China must keep up efforts...
1  fake  Donald Trump likes to promote himself a shrewd...
2  true  JOHANNESBURG (Reuters) - A South African High ...
3  true  WASHINGTON (Reuters) - President Donald Trump ...
4  fake  21st Century Wire says There is an amazing sce...
5  true   NEW YORK (Reuters) - A judge will decide by t...
6  fake  Bill Clinton was falsely confident in his plan...
7  fake  Nassau County cops busted a Long Island man wh...
8  fake  With 28 days left to go until election day, a ...
9  fake  Putin Tells Russian Army To Prepare For WWWIII...


,label,text
0,true,"[beijing, reuters, china, must, keep, efforts,..."
1,fake,"[donald, trump, likes, promote, shrewd, busine..."
2,true,"[johannesburg, reuters, south, african, high, ..."
3,true,"[washington, reuters, president, donald, trump..."
4,fake,"[century, wire, says, amazing, scene, unfoldin..."
5,true,"[new, york, reuters, judge, decide, end, month..."
6,fake,"[bill, clinton, falsely, confident, plan, end,..."
7,fake,"[nassau, county, cops, busted, long, island, m..."
8,fake,"[days, left, go, election, day, new, poll, nbc..."
9,fake,"[putin, tells, russian, army, prepare, wwwiii,..."


In [14]:
# Convert messages (as lists of string tokens) to strings
data_shuffled["text"] = data_shuffled["text"].agg(lambda x: " ".join(map(str, x)))
data_shuffled.head()

# Initialize count vectorizer
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(data_shuffled["text"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

# Convert strings to vectors using BoW
dataset1_bow = bow_transformer.transform(data_shuffled["text"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {dataset1_bow.shape}")
print(f"Amount of non-zero occurrences: {dataset1_bow.nnz}")



20 BOW Features: ['aaja', 'aalberg', 'aalberts', 'aaldef', 'aaliyah', 'aalo', 'aamer', 'aamin', 'aaminus', 'aammir', 'aamom', 'aampampacbreturn', 'aampampbcbparsefloatastringbaeffunctionavar', 'aampexkamp', 'aan', 'aapa', 'aapi', 'aapl', 'aaplo', 'aaplus']
Total number of vocab words: 198902
Shape of sparse matrix: (44869, 198902)
Amount of non-zero occurrences: 7113367


In [15]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(dataset1_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(dataset1_bow)
print(messages_tfidf.shape)

# Convert spam and ham labels to 0 and 1 (or, vice-versa)
FactorResult = pd.factorize(data_shuffled["label"])
data_shuffled["label"] = FactorResult[0]
data_shuffled.head()

(44869, 198902)


,label,text
0,0,beijing reuters china must keep efforts revolu...
1,1,donald trump likes promote shrewd businessman ...
2,0,johannesburg reuters south african high court ...
3,0,washington reuters president donald trump spea...
4,1,century wire says amazing scene unfolding amer...


In [20]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, data_shuffled["label"], test_size=0.2
)

print(f"train dataset features size: {msg_train.shape}")
print(f"train dataset label size: {label_train.shape}")

print(f"test dataset features size: {msg_test.shape}")
print(f"test dataset label size: {label_test.shape}")

# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)


# Make predictions
predict_train = clf.predict(msg_train)

print(
    f"Accuracy of Train dataset: {metrics.accuracy_score(label_train, predict_train):0.3f}"
)

# an example prediction
print(
    "predicted:",
    clf.predict(
        tfidf_transformer.transform(bow_transformer.transform([data_shuffled["text"][9]]))
    )[0],
)
print("expected:", data_shuffled["label"][9])

# print the overall accuracy of the model
label_predictions = clf.predict(msg_test)
print(f"Accuracy of the model: {metrics.accuracy_score(label_test, label_predictions):0.3f}")

train dataset features size: (35895, 198902)
train dataset label size: (35895,)
test dataset features size: (8974, 198902)
test dataset label size: (8974,)
Accuracy of Train dataset: 1.000
predicted: 1
expected: 1
Accuracy of the model: 0.997


In [65]:
T1 = """TOKYO (Reuters) - Britain and Japan said on Thursday they would cooperate in countering the threat posed by North Korea, two days after it fired a missile over northern Japan, and will call on China to exert its leverage. Prime Minister Theresa May, looking to strengthen relations with one of her closest allies ahead of Brexit, is visiting Japan as it responds to an increasing military threat. Terming North Korea s missile program  a global threat , Prime Minister Shinzo Abe told a news conference that Japan and Britain would cooperate.  It is very meaningful that Prime Minister May and I agreed to further strengthen pressure on North Korea and to call on China to play a larger role,  he added. May agreed, noting that China, North Korea s lone major ally, had been involved in U.N. Security Council debate earlier this week.   China does have a particular position in this, they have leverage on North Korea and I believe we should be encouraging China to exercise that leverage to do what we all want - which is to ensure that North Korea is not conducting these illegal acts.  May toured Japan s flagship Izumo helicopter carrier for a military briefing with Minister of Defence Itsunori Onodera before attending a national security meeting. May and Abe agreed on a joint declaration on security cooperation, including plans for British soldiers to take part in military exercises on Japanese soil and for collaboration to address the threat of cyber and militant attacks when Japan hosts the Olympics in 2020.  North Korea featured heavily in the talks after it launched a ballistic missile on Tuesday that passed over Japanese territory, prompting international condemnation.  May s office had said the two leaders were expected to discuss the possibility of further sanctions on North Korea, but neither Abe nor May touched on the issue at the news conference. The Global Times, a publication of the official People s Daily of China s ruling Communist Party, criticized an earlier comment of May s comment calling for more pressure from China.  Beijing does not need London to teach it how to deal with North Korea,  the newspaper said. Asked about the United States, Japan and Britain looking to impose new sanctions on North Korea, Chinese Foreign Ministry spokeswoman Hua Chunying said the situation could only be resolved peacefully through dialogue.  We think it is regrettable that some countries selectively overlook the relevant Security Council resolutions  demand to advance dialogue, and stubbornly emphasize pressure and sanctions,  she told a daily news briefing.  OUTWARD-LOOKING  Apart from security, May s trip has focused on trade and investment. She is keen to convince nervy investors that Britain s exit from the European Union will not make it a less attractive business partner. Both May and Abe addressed a delegation of British business leaders and senior representatives from major Japanese investors in Britain, such as carmakers Nissan, Toyota and conglomerate Hitachi. Abe told the gathering that May had assured him Britain s negotiations on leaving the European Union would be transparent. May said Japanese investment after Britain s vote to leave the EU was a vote of confidence and she pledged to build close trade ties with Japan.  I very much welcome the commitment from Japanese companies such as Nissan, Toyota, Softbank and Hitachi,  May said.  I am determined that we will seize the opportunity to become an ever more outward-looking global Britain, deepening our trade relations with old friends and new allies.   During a two-hour train ride between Kyoto and Tokyo late on Wednesday, the two leaders discussed Brexit, with May talking Abe through the details of a series of papers published in recent weeks setting out her negotiating position. May said on Wednesday Japan s upcoming trade deal with the EU could offer a template for a future Japan-Britain trade agreement, the latest attempt to show investors that Brexit will not lead to an overnight change in business conditions. Japan has been unusually open about its concerns over Brexit, worrying that 40 billion pounds ($51.68 billion) of Japanese investment in the British economy could suffer if trading conditions change abruptly when Britain leaves the bloc."""

T2 = '''NEW DELHI (Reuters) - Indian Prime Minister Narendra Modi plans to reshuffle his cabinet over the next two days and some mid-level ministers have offered to quit to pave the way for changes, sources in the party and aides to the ministers said. A revamp of the cabinet has been on the cards for months because some ministers are holding multiple portfolios which is seen as a drag on efficiency. Modi is also seeking to improve governance before he seeks re-election in 2019.     With economic growth slowing to its slowest pace in three years, Modi is coming under pressure to deliver the promises he made in 2014.     There was no official announcement of the cabinet revamp but  sources Reuters spoke to said if it took place it would have to be done before Modi leaves on a foreign tour on Sunday.     The death of Environment Minister Anil Madhav Dave and the election of Minister for Urban Development M. Venkaiah Naidu as vice-president opened vacancies, giving Modi an opportunity to bring in members from regional political parties.     According to four party sources, Sanjeev Balyan, minister of state for water resources, Rajiv Pratap Rudy, minister of state for skill development and entrepreneurship, Kalraj Mishra, minister of micro, small and medium enterprises, and Mahendra Nath Pandey minister of state for human resource development have all offered to resign.'''


T3 = '''Brooklyn College is advising New York City police officers to use bathrooms on the far end of campus in order to avoid being seen by students who might be offended by their presence.The Excelsior, the college is student newspaper, reported last week that Director of Public Safety Donald Wenz would  prefer  if members of the New York Police Department (NYPD) used facilities in the West End Building (WEB)  rather than walking across either quad to use the bathroom. Wenz is comments come in the wake of a film screening of the movie  Watched,  which documents an NYPD informant who surveilled Muslim students on campus for a period of four years as part of the city is counterterrorism efforts. After the screening, students were allowed to share their thoughts on having NYPD officers on campus. I disagree with them being on campus. Especially allowing them to use the building where student groups are held,  one unidentified student remarked, according to The Excelsior, later noting that he would be sending a petition to College President Michelle Anderson, pressuring her to issue a statement  that we do not want the NYPD on campus in any respect even if it is just to take breaks and use bathrooms. After discovering The Excelsior is report, The New York Post took a trip to the WEB, where officers are now encouraged to use the bathroom, and found an  out of order  sign on the stall door, with a dirty and broken toilet on the inside. The bathroom is horrendous. You can only wash your hands in one of the sinks because the other two are broken,  one student told the Post, though others argued that NYPD should not be allowed on campus at all. I know students from every background and across every major. They don t feel comfortable around cops. They just don t. It makes safe spaces feel not so safe,  one unidentified student remarked, with another telling the Post that it is  weird  seeing cops on campus.But members of the NYPD who monitor the neighborhood surrounding the school told the Post that the student sentiments are  insane.    Campus Reform '''



In [67]:
def predict_text(text):
    dict = {'label':['unknown'],
        'text':[text],
       }
    WP = pd.DataFrame(dict)
    WP["text"] = WP["text"].apply(text_preprocess)

    # Convert messages (as lists of string tokens) to strings
    WP["text"] = WP["text"].agg(lambda x: " ".join(map(str, x)))
    
    data_to_eval = WP["text"][0]
    return clf.predict(tfidf_transformer.transform(bow_transformer.transform([data_to_eval])))[0]

print(predict_text(T3))    

1


For predictions 

True >> 0

False >> 1

In [70]:
#clf.save_model('./CLF_model.json')
import pickle
pickle.dump(clf, open("./CLF_model.json", "wb"))


OSError: [Errno 9] Bad file descriptor